In [3]:
import sys
import pandas as pd
import datetime as dt
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

c:\Users\Ronak.pandya\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
def fuzzy_check(df_source,df_reference):
    df_source['fuzzy_check_source']=df_source["BRAND"]+"-"+df_source["SUB_BRAND"]
    df_reference['fuzzy_check_reference']=df_reference["PROD_BRND"]+"-"+df_reference["PROD_SUB_BRND"]
    cat_list = list(set(list(df_source["CATEGORY"])))
    source_list_temp=list()
    final_brand_subbrand_list=list()
    for i in cat_list:
        df_source_v1 = df_source.loc[df_source['CATEGORY'] == i]
        df_reference_v1=df_reference.loc[df_reference['PROD_CTGY'] == i]
        source_list = list(df_source_v1['fuzzy_check_source'])
        ref_list=list(set(list(df_reference_v1['fuzzy_check_reference'])))
        for i in source_list:
            try:
                if i in ref_list:
                    source_list_temp.append(i)
                elif i not in ref_list:
                    for j in ref_list:
                        if fuzz.WRatio(i, j)>=95:
                            source_list_temp.append(j)
                        else:
                            pass
                
                    source_list_temp.append('NO MATCH')
            except Exception as e:
                print(e)
    df_source['brand_check']=source_list_temp
    return df_source

In [5]:
def create_brand_code(df_source,df_reference):
    df_reference['fuzzy_check_reference']=df_reference["PROD_BRND"]+"-"+df_reference["PROD_SUB_BRND"]
    df_final = pd.merge(df_source,df_reference,left_on=['CATEGORY','brand_check'],\
                                  right_on=['PROD_CTGY','fuzzy_check_reference'])[:]
    df_final['Week_1']=df_final['WEEK'].dt.strftime('%d-%m-%Y')
    return df_final

In [6]:
def create_date_table2(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df['Date_ref'] = df['Date'].dt.strftime('%d-%m-%Y')
    df["Day"] = df['Date'].dt.day_name()
    df["Week"] = df.Date.dt.isocalendar().week
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    df.insert(0, 'date_id', range(1, 1 + len(df)))
    return df

In [7]:
def create_cntry_master(df_input):
    country_list = list(set(list(df_input['MARKET'])))
    df_cntry= pd.DataFrame()
    df_cntry['cntry']=country_list
    df_cntry.insert(0,'cntry_id',range(1,1+len(df_cntry)))
    return df_cntry

In [8]:
def create_brand_master(df_input):
    brand_list = list(set(list(df_input['BRAND'])))
    df_brand= pd.DataFrame()
    df_brand['brand']=brand_list
    df_brand.insert(0,'brand_id',range(1,1+len(df_brand)))
    return df_brand

In [9]:
def create_sub_brand_master(df_input,df_brand):
    sub_brand_list = list(df_input['SUB_BRAND'])
    brand_list= list(df_input['BRAND'])
    brand_code_list = list(df_input['BRAND_CODE'])
    df_sub_brand = pd.DataFrame()
    df_sub_brand['sub_brand']=sub_brand_list
    df_sub_brand['brand']=brand_list
    df_sub_brand['brand_code']=brand_code_list
    df_sub_brand_int = pd.merge(df_sub_brand,df_brand,left_on="brand",right_on='brand')[['sub_brand','brand_id',\
                                                                                         'brand_code']]
    df_sub_brand_int.insert(0,'sub_brand_id',range(1,1+len(df_sub_brand_int)))
    return df_sub_brand_int
    

In [10]:
def create_cat_master(df_input):
    cat_list = list(set(list(df_input['CATEGORY'])))
    df_cat= pd.DataFrame()
    df_cat['category']=cat_list
    df_cat.insert(0,'cat_id',range(1,1+len(df_cat)))
    return df_cat

In [11]:
def create_sub_cat_master(df_input,df_cat):
    sub_cat_list = list(df_input['SUB_CATEGORY'])
    cat_list= list(df_input['CATEGORY'])
    df_sub_cat_master = pd.DataFrame()
    df_sub_cat_master['sub_category']=sub_cat_list
    df_sub_cat_master['category']=cat_list
    df_sub_cat_master_int = pd.merge(df_sub_cat_master,df_cat,left_on ="category",right_on="category")[['sub_category',\
                                                                                                        'cat_id']]
    df_sub_cat_master_int=df_sub_cat_master_int.drop_duplicates()
    
    df_sub_cat_master_int.insert(0,'sub_cat_id',range(1,1+len(df_sub_cat_master_int)))
    return df_sub_cat_master_int

In [12]:
def create_seg_master(df_input):
    seg_master_list = list(set(list(df_input['SEGMENT'])))
    df_segment_mstr = pd.DataFrame()
    df_segment_mstr['segment']=seg_master_list
    df_segment_mstr.insert(0,'seg_id',range(1,1+len(df_segment_mstr)))
    return df_segment_mstr

In [13]:
def create_sub_seg_mstr(df_input,df_segment_mstr):
    seg_list = list(df_input['SEGMENT'])
    sub_seg_list = list(df_input['SUB_SEGMENT'])
    df_sub_seg = pd.DataFrame()
    df_sub_seg['segment']=seg_list
    df_sub_seg['sub_segment']=sub_seg_list
    df_sub_segment_int = pd.merge(df_sub_seg,df_segment_mstr,left_on="segment",right_on="segment")[['sub_segment',\
                                                                                                    'seg_id']]
    df_sub_segment_int.insert(0,'sub_seg_id',range(1,1+len(df_sub_segment_int)))
    return df_sub_segment_int

In [14]:
def create_fact(df_input,df_sub_segment,df_sub_cat,df_sub_brand,df_cntry,df_date):
    
    df_fact_int=pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(df_input,df_sub_segment,left_on ="SUB_SEGMENT",right_on="sub_segment"),\
                         df_sub_cat,left_on="SUB_CATEGORY",right_on="sub_category"),df_sub_brand,left_on="SUB_BRAND",\
                         right_on="sub_brand"),df_cntry,left_on="MARKET",right_on="cntry"),df_date,left_on="Week_1",\
                         right_on='Date_ref')[['ITEM_ID','SUM_STOR_CNT','SUM_SLS_UNITS','SUM_SLS_VAL_LCLCCY_AMT',\
                                               'SUM_SLS_VOL','SUM_TOT_DSTRBTN_PTS','DISTR_WTD_VOL','sub_seg_id','seg_id',\
                                              'cat_id','sub_cat_id','sub_brand_id','brand_id','brand_code','cntry_id',\
                                              'date_id']]
    df_fact_int.insert(0,'fact_id',range(1,1+len(df_fact_int)))
    return df_fact_int

In [17]:
def main():
    df_source = pd.read_excel(r'C:\Users\Ronak.pandya\Downloads\poc_data_raw.xlsx')
    df_reference= pd.read_csv(r'C:\Users\Ronak.pandya\Downloads\store_mapping_sample_fr.csv')
    print("\n Run Fuzzy check for brand & sub Brand codes")
    df_source = fuzzy_check(df_source,df_reference)
    print("\n----------------------------------------------------")
    print("\n Fuzzy Check for brand and Sub Brand Codes Completed")
    df_input= create_brand_code(df_source,df_reference)
    print("\n The source DataFrame has been created with Week and Product Brand code")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Date Master"))
    df_date = create_date_table2()
    print("\n Data Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Region Master"))
    df_cntry = create_cntry_master(df_input)
    print("\n Region Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the master Table:{0}".format("Brand Master"))
    df_brand = create_brand_master(df_input)
    print("\n Brand Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Sub Brand Master"))
    df_sub_brand=create_sub_brand_master(df_input,df_brand)
    print("\n Sub Brand Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Catgory Master"))
    df_cat=create_cat_master(df_input)
    print("\n Category Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Sub Catgory Master"))
    df_sub_cat=create_sub_cat_master(df_input,df_cat)
    print("\n Sub Category Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("segment Master"))
    df_segment_mstr=create_seg_master(df_input)
    print("\n Segment Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("sub segment Master"))
    df_sub_segment = create_sub_seg_mstr(df_input,df_segment_mstr)
    print("\n Sub Segment Master Created")
    print("\n----------------------------------------------------")
    print("\n Building the Master Table:{0}".format("Fact"))
    df_fact= create_fact(df_input,df_sub_segment,df_sub_cat,df_sub_brand,df_cntry,df_date)
    print("\n Fact Table Created")
    print("\n----------------------------------------------------")
    print("\n Process has completed")
    # return df_fact.to_json('temp.json', orient='records', lines=True)

if __name__=="__main__": 
    main() 


 Run Fuzzy check for brand & sub Brand codes

----------------------------------------------------

 Fuzzy Check for brand and Sub Brand Codes Completed

 The source DataFrame has been created with Week and Product Brand code

----------------------------------------------------

 Building the master Table:Date Master

 Data Master Created

----------------------------------------------------

 Building the master Table:Region Master

 Region Master Created

----------------------------------------------------

 Building the master Table:Brand Master

 Brand Master Created

----------------------------------------------------

 Building the Master Table:Sub Brand Master

 Sub Brand Master Created

----------------------------------------------------

 Building the Master Table:Catgory Master

 Category Master Created

----------------------------------------------------

 Building the Master Table:Sub Catgory Master

 Sub Category Master Created

--------------------------------------

In [20]:

import sys

def sum(firstVar,secondVar):
    print(firstVar,secondVar)
    # return firstVar + secondVar

# print(sum(sys.argv[1],sys.argv[2]))


def main(firstVar,secondVar):
    sum(firstVar,secondVar)

if __name__=="__main__": 
    main(1,2) 

# sys.stdout.flush()

1 2
